In [1]:
import tensorwrap as tf
import jax
from jax import jit

Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!


In [2]:
x = tf.range(1, 1e5)
x = tf.reshape(x, (1, -1))
y = x + 10

In [3]:
model = tf.nn.Sequential([
    tf.nn.layers.Dense(10),
    tf.nn.layers.Dense(1)
])
model.compile(
    loss=tf.nn.losses.mae,
    optimizer=tf.nn.optimizers.gradient_descent(learning_rate = 1e-2),
)

In [4]:
vars(model.layers[0])

{'trainable_variables': {},
 'dtype': None,
 'kwargs': {},
 'built': False,
 'name': 'layer0',
 'units': 10,
 'activation': None,
 'use_bias': True,
 'kernel_initializer': 'glorot_uniform',
 'bias_initializer': 'zeros',
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None,
 'dynamic': False}

In [5]:
jax.tree_util.tree_leaves(model)

In [6]:
jax.tree_flatten(model)

/tmp/ipykernel_11515/3410477249.py:1: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  jax.tree_flatten(model)


([<tensorwrap.nn.models.Sequential at 0x7f82912e3f70>], PyTreeDef(*))

In [7]:
jax.tree_util.tree_structure(model)

PyTreeDef(*)

In [8]:
%timeit model.fit(x, y, epochs=100, verbose=0)

146 ms ± 3.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
model(x)

Array([[6566418.]], dtype=float32)